# fluid 的数据输入方式

## 例1 直接输入数据  

CSY 2020-1-27  
简单起见，以一个线性的全连接层为例，没有设计标签输入和反向网络  
程序另存为direct_feed.py  
本例已收入博客 https://blog.csdn.net/aLife2P6/article/details/104092231

###  加载必要的软件包

In [1]:
import paddle.fluid as fluid
import numpy

### 产生训练数据--给网络准备食物 

神经网络挑食，要求数据是Tensor，最简单的食物就是Numpy的Array数组。  
参考网址：https://www.paddlepaddle.org.cn/documentation/docs/zh/user_guides/howto/basic_concept/tensor.html#tensor  
神经网络不仅挑食材，还挑口味, 主要是float32。详见：https://www.paddlepaddle.org.cn/documentation/docs/zh/user_guides/howto/prepare_data/feeding_data.html#fluid

In [2]:
train_data=numpy.array([[1.0],[2.0],[3.0],[4.0]]).astype('float32') # 食物的shape=(4,1)，type='float32'
print(train_data.shape)

(4, 1)


### 配置输入接口--设计网络的嘴巴  

嘴巴的大小和形状要和喂入的食物吻合  
shape中的None表示可以是任意正整数。 
None可以用-1代替，作用相同； 
本例中None也可以用4代替，这样和训练的食物相同，但是以后用模型预测时，食物的shape就只能是[4,1]

In [3]:
x = fluid.data(name="x",shape=[None,1],dtype='float32') # 嘴巴的shape容纳了食物的shape。

### 配置网络--设计一个输出的全连接层

对应一个x输入，网络就会产生一个预测值，预测值y_predict=ax,  
此处的权重a由初始化产生，训练时自动调整。  
由于本网络省去了反向网络，没有学习能力，a只与初始化程序有关。

In [4]:
y_predict = fluid.layers.fc(input=x,size=1,act=None)

### 初始化参数

In [5]:
cpu = fluid.core.CPUPlace() # 在cpu上操作
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program()) # 运行执行器初始化网络参数（采用默认的初始化程序）

[]

### 开始训练

训练的过程就是运行主程序的过程。  
由于没有反向网络，多次训练结果没有变化。  
我的环境下的a大约为-0.65

In [12]:
for i in range(3): 
    outs = exe.run( # 加载主程序运行执行器
        feed={'x':train_data}, # 从名为x的嘴巴喂入train_data食物
        fetch_list=[y_predict])
    # 每次训练的结果
    print(outs) # 输出列表仅有一个内容，就是out[0]=y_predict
# 训练到最后一次的结果
#print(outs) # 输出列表仅有一个内容，就是out[0]=y_predict

[array([[-0.6521425],
       [-1.304285 ],
       [-1.9564276],
       [-2.60857  ]], dtype=float32)]
[array([[-0.6521425],
       [-1.304285 ],
       [-1.9564276],
       [-2.60857  ]], dtype=float32)]
[array([[-0.6521425],
       [-1.304285 ],
       [-1.9564276],
       [-2.60857  ]], dtype=float32)]


## 例2

CSY 2020-1-29

PaddlePaddle根据Reader返回的数据形式的不同，可分为Batch级的Reader和样本级的Reader。  
参考网址：https://www.paddlepaddle.org.cn/documentation/docs/zh/user_guides/howto/prepare_data/prepare_steps.html#user-guide-prepare-steps